# Social Sensing: Ejemplo Práctico

Montamos la carpeta en el Drive para leer los datos

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Colab\ Notebooks/SII-UPV/

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/SII-UPV


### 1) Cargar y explorar los tweets

El dataframe se compone de las siguientes columnas:
* id: Identificador del tweet.
*	user: Nombre del usuario que publicó el tweet.
*	fecha: Fecha de publicación.
*	idioma: Idioma del tweet.
*	texto: El contenido del tweet.
*	retweeted, replyed, liked: Datos relacionados con interacciones (si fue retuiteado, respondido o gustado).


In [2]:
# Importamos las librerías necesarias
import pandas as pd

# Cargar el archivo CSV
file_path = 'datos/torrevieja_tweets_sample.csv'
df = pd.read_csv(file_path)
print(f'Filas del dataframe: {len(df)}')
# Mostrar una vista previa de los datos
df.head()

Filas del dataframe: 2000


,id,user,fecha,idioma,texto,retweeted,replyed,liked
0,153063041880162305,José Gosálvez,2011-12-31T10:40:47.000Z,es,Vamos a La Zenia (cerca de Torrevieja) a pasar...,0.0,0.0,0.0
1,370998523946610688,Andrea Cortés,2013-08-23T19:58:35.000Z,es,Ya salimos para torrevieja.,0.0,0.0,0.0
2,387193471263842304,Xema Cintas,2013-10-07T12:31:31.000Z,es,"Sesión de enfermería en videoconferencia,@Vina...",2.0,0.0,1.0
3,353483075994402818,Alve,2013-07-06T11:58:27.000Z,es,Desde muy cerquita de Torrevieja donde estuve ...,1.0,1.0,0.0
4,1197128195172130816,DERBORAH G. PEREIRA,2019-11-20T12:22:57.000Z,es,Ui que miedo dos Nazistas ui asim mi gusta kkk...,0.0,0.0,0.0


### 2) Preprocesamiento de los textos

El próximo paso será preprocesar el texto. Nos enfocaremos en limpiar los tweets quitando URLs, signos de puntuación, menciones, hashtags, emojis, stopwords y convertir el texto a minúsculas.

In [3]:
import re
import nltk
from nltk.corpus import stopwords

# Descargar las stopwords en español si es necesario
nltk.download('stopwords')

# Lista de stopwords en español
stop_words = set(stopwords.words('spanish'))

# Función de preprocesamiento para limpiar el texto
def clean_text(text):
    text = text.lower()  # Convertimos a minúsculas
    text = re.sub(r"http\S+|www\S+", '', text)  # Quitamos URLs
    text = re.sub(r'\@\w+|\#', '', text)  # Quitamos menciones y hashtags
    text = re.sub(r'[^\w\s]', '', text)  # Quitamos puntuación y emoticonos
    text = re.sub(r'torrevieja','', text) # Quitamos las palabras empleadas para recopilar los mensajes
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Aplicamos la limpieza de texto a la columna de 'texto'
df['texto_limpio'] = df['texto'].apply(clean_text)

# Eliminar los mensajes que estén vacíos después del preprocesamiento
df = df[df['texto_limpio'].str.strip() != '']

# Mostrar una vista previa del texto limpio
df[['texto', 'texto_limpio']].head(10)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,texto,texto_limpio
0,Vamos a La Zenia (cerca de Torrevieja) a pasar...,vamos zenia cerca pasar fin año costumbre hace...
1,Ya salimos para torrevieja.,salimos
2,"Sesión de enfermería en videoconferencia,@Vina...",sesión enfermería videoconferencia cuidados tr...
3,Desde muy cerquita de Torrevieja donde estuve ...,cerquita contigo dias desearte feliz dia feliz...
4,Ui que miedo dos Nazistas ui asim mi gusta kkk...,ui miedo dos nazistas ui asim gusta kkkkdeport...
5,NUEVA EDICIÓN DE LA UNIVERSIDAD PERMANENTE DE ...,nueva edición universidad permanente ua tres a...
6,Una cena con ellos siempre viene bien. #famili...,cena siempre viene bien familia
7,Aldembrau 🐬🐋💪🏻💪🏻 @ Torrevieja https://t.co/oMH...,aldembrau
8,Yo quiero sentir tocar las nubes..Yo quiero ba...,quiero sentir tocar nubesyo quiero bailar aman...
9,Guapas por Torrevieja✌ Calas bonitas que encon...,guapas calas bonitas encontramos palmeras playa


In [ ]:
print(f'Filas del dataframe tras preprocesar: {len(df)}')

Filas del dataframe tras preprocesar: 1936


### 3) Análisis de sentimiento

Una vez los textos han sido limpiados correctamente, eliminando URLs, menciones, hashtags, puntuación, y emojis, podemos proceder a extraer el sentimiento de las publicaciones. Vamos a utilizar un modelo preentrenado de análisis de sentimiento multilingüe. Utilizaremos un modelo de la librería de Transformers de Huggingface, específico para análisis de sentimiento en español, como `bert-base-multilingual-uncased-sentiment`. Es recomendable cargar el modelo en GPU. Para ello cambiamos el tipo de entorno de ejecución (T4 GPU).


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoModelForTokenClassification, AutoTokenizer, pipeline
import torch
import warnings

warnings.filterwarnings('ignore')

# Verificar si hay GPU disponible
device = 0 if torch.cuda.is_available() else -1
#print(device)
# Cargamos el pipeline de análisis de sentimiento en GPU si está disponible

model_name = "pysentimiento/robertuito-sentiment-analysis"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

sa_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=device)


# Aplicamos el análisis de sentimiento a los textos limpios
df['sentimiento'] = df['texto_limpio'].apply(lambda x: sa_pipeline(x)[0]['label'])


config.json:   0%|          | 0.00/925 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/435M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Mostramos algunos ejemplos de mensajes con sentimiento positivo y negativo:

In [ ]:
# Filtrar los mensajes positivos y negativos
mensajes_positivos = df[df['sentimiento'] == 'POS']
mensajes_negativos = df[df['sentimiento'] == 'NEG']

# Ejemplos de mensajes positivos
print("Ejemplos de mensajes positivos:\n")
for msg in mensajes_positivos['texto'].head():
    print(f"- {msg}\n")

# Ejemplos de mensajes negativos
print("\nEjemplos de mensajes negativos:\n")
for msg in mensajes_negativos['texto'].head():
    print(f"- {msg}\n")

Ejemplos de mensajes positivos:

- Desde muy cerquita de Torrevieja donde estuve contigo unos dias desearte un feliz dia y un feliz cumpleaños! Te deseo lo mejor @SofiHect26

- Una cena con ellos siempre viene bien. #familia en Torrevieja https://t.co/zbAayWjIDP

- Guapas por Torrevieja✌ Calas bonitas que encontramos😄 #Palmeras #Playa… https://t.co/ZGDVcAAtks

- Que gira más bonita nos están dejando @SweetCalifornia 🤩 Próximas fechas:-19 de Octubre #Sevilla (Entradas Agotadas)-27 de Octubre #Pamplona -16 de Noviembre #SantCugat -23 de Noviembre #Torrevieja #SweetCalifornia #GiraOrigen2019 #Origen #Gira #Concierto #LoveMusic 💜 https://t.co/b8wc9cDaak

- Buen día en Torrevieja 👌


Ejemplos de mensajes negativos:

- Ui que miedo dos Nazistas ui asim mi gusta kkkkDeportame kkkkkkk Vai te toma por culo. en Playa Torrevieja https://t.co/BEydIHo0HU

- En Torrevieja detenidas las dueñas de 3 perritos encerrados en el coche a pleno sol, 1 murió. http://t.co/CSeY15QRbN

- @Magoacme Tu tb eres de

### 4) Reconocimiento de entidades nombradas
En esta sección, vamos a aplicar un modelo de Reconocimiento de Entidades Nombradas (NER) a los tweets para identificar automáticamente entidades clave como personas, organizaciones, y lugares. El NER nos permitirá extraer esta información directamente desde los textos, lo cual es útil para comprender qué entidades son mencionadas con más frecuencia en relación con los temas tratados en los tweets. Utilizaremos un modelo preentrenado que funciona en varios idiomas, incluido el español, para realizar esta tarea.

In [ ]:
# Cargamos el pipeline de NER
ner_model_name = "xlm-roberta-large-finetuned-conll03-english"
model = AutoModelForTokenClassification.from_pretrained(ner_model_name)
tokenizer = AutoTokenizer.from_pretrained(ner_model_name)

ner_pipeline = pipeline("token-classification", model=model, aggregation_strategy="simple", tokenizer=tokenizer, device=device)

# Aplicar NER a algunos tweets como ejemplo
df['entidades'] = df['texto'].apply(lambda x: ner_pipeline(x))

# Mostrar algunos ejemplos de entidades nombradas en los tweets
print("\nEjemplos de entidades nombradas:\n")
for i, row in df[['texto', 'entidades']].head().iterrows():
    entities = [{"text": str(entity["word"]), "label": entity["entity_group"], "score": entity["score"]} for entity in row['entidades']]
    print(f"Tweet: {row['texto']}")
    print(f"Entidades: {entities}\n")


config.json:   0%|          | 0.00/852 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-large-finetuned-conll03-english were not used when initializing XLMRobertaForTokenClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Device set to use cuda:0



Ejemplos de entidades nombradas:

Tweet: Vamos a La Zenia (cerca de Torrevieja) a pasar el fin de año como ya es costumbre desde hace unos cuantos años.
Entidades: [{'text': 'La Zenia', 'label': 'LOC', 'score': np.float32(0.9999924)}, {'text': 'Torrevieja', 'label': 'LOC', 'score': np.float32(0.99998695)}]

Tweet: Ya salimos para torrevieja.
Entidades: [{'text': 'torrevieja', 'label': 'LOC', 'score': np.float32(0.9999888)}]

Tweet: Sesión de enfermería en videoconferencia,@Vinaloposalud y @Dpto_Torrevieja,cuidados #traqueostomía http://t.co/I9psg3E53w
Entidades: [{'text': '@Vinaloposalud', 'label': 'ORG', 'score': np.float32(0.9693107)}, {'text': '@Dpto_Torrevieja', 'label': 'ORG', 'score': np.float32(0.9923077)}]

Tweet: Desde muy cerquita de Torrevieja donde estuve contigo unos dias desearte un feliz dia y un feliz cumpleaños! Te deseo lo mejor @SofiHect26
Entidades: [{'text': 'Torrevieja', 'label': 'LOC', 'score': np.float32(0.9999967)}, {'text': 'SofiHect26', 'label': 'PER', 'scor

**Ejercicio 1:** Calcular el total y porcentaje de tweets positivos, neutros y negativos.

**Ejercicio 2:** Generar una nube de palabras para cada categoría de sentimiento (positivos, negativos, neutros).

**Ejercicio 3:** Extrer las 10 entidades nombradas más frecuentes en los textos.